# Agents

In [1]:
from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [2]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Hello! How are you?")
result.final_output

'HELLO! I’M GREAT, THANK YOU! HOW ABOUT YOU?'

In [3]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
)

topic = "Boogers"
result = await Runner.run(joke_agent, topic)
result.final_output

"Why did the booger stay in school?\n\nBecause it couldn't stop picking up things!"

In [4]:
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in a different language.",
)

joke_result = await Runner.run(joke_agent, topic)
language = "French"
translated_result = await Runner.run(language_agent, f"Translate this joke to {language}: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

Original joke:
Why did the booger apply for a job?

Because it wanted to get picked!

Translated joke:
Pourquoi la crotte de nez a-t-elle postulé à un emploi ?

Parce qu'elle voulait se faire choisir !


# Structured Output

Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for tasks that require specific formatting or data extraction.

In [5]:
from pydantic import BaseModel
from agents import Agent

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int # in minutes
    servings: int

recipe_agent = Agent(
    name="Recipe Agent", 
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job"
                  " is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "carbonara")
recipe = response.final_output

In [6]:
recipe

Recipe(title='Classic Spaghetti Carbonara', ingredients=['400g spaghetti', '150g pancetta or guanciale, diced', '2 large eggs', '2 large egg yolks', '100g Parmesan cheese, grated', 'Salt, to taste', 'Freshly ground black pepper, to taste', '2 cloves garlic, peeled and crushed (optional)'], cooking_time=25, servings=4)

# Tool Calling
Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs. This can be useful for tasks that require access to external data or services.

In [7]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return "sunny"

@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting temperature for {city}")
    return "70 degrees"

agent = Agent(
    name="Weather Agent",
    instructions="You are the local weather agent. You are given a city and you need to tell the weather and temperature. For any unrelated queries, say I cant help with that.",
    tools=[get_weather, get_temperature]
)

result = await Runner.run(agent, "Dallas")
result.final_output

Getting weather for Dallas
Getting temperature for Dallas


'The weather in Dallas is sunny, and the temperature is 70 degrees.'

In [8]:
from agents import WebSearchTool

news_agent = Agent(
    name="News Reporter",
    instructions="You are a news reporter. Your job is to find recent news articles on the internet about US politics.",
    tools=[WebSearchTool()]
)

result = await Runner.run(news_agent, "find news")
print(result.final_output)

Here are some recent developments in U.S. politics:

**Rise in Political Violence**

The United States is witnessing a concerning increase in political violence. Recently, Democratic Minnesota state Rep. Melissa Hortman and her husband were fatally shot, and state Sen. John Hoffman and his wife were seriously wounded by an individual impersonating a police officer. This incident is part of a broader trend, with the U.S. Capitol Police reporting nearly 9,500 threats against public officials in 2024 alone. A Reuters investigation identified at least 51 incidents of political violence from January to October 2024, marking the highest surge since the 1970s. ([time.com](https://time.com/7294891/political-violence-rise-america/?utm_source=openai))

**New Visa Requirements for Foreign Students**

The U.S. State Department has resumed student visa processing with a new mandate requiring applicants to make their social media accounts public for government review. This policy is part of a broade